# Trajectory analysis notebook - density profile
In this notebook we show how the MDAnalysis package can be used to perform some _basic_ analysis of the simulations using the atomic trajectory generated by openMM (or any other MD package).

In [1]:
# Packages for the trajectory analysis and visualisation
import MDAnalysis as md
import nglview as ng

# python packages
import pandas as pd                  # Dataframes and reading CSV files
import numpy as np                   # Numerical libraries
import matplotlib.pyplot as plt      # Plotting library
from lmfit import Model              # Least squares fitting library
from scipy.optimize import curve_fit # Alternative curve fittting library

/Users/237454k/miniconda3/envs/work/lib/python3.9/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")
/Users/237454k/miniconda3/envs/work/lib/python3.9/site-packages/MDAnalysis/coordinates/TRJ.py:1209: DeprecationWarning: Please use `netcdf_file` from the `scipy.io` namespace, the `scipy.io.netcdf` namespace is deprecated.
  class NCDFPicklable(scipy.io.netcdf.netcdf_file):


For simplicity we define three variables that represent the three Cartesian directions. 

In [2]:
X = 0
Y = 1
Z = 2

First of all we have to load the trajectory. In this case the trajectory was written in the _CHARMM_ DCD format, which is binary (not ASCII) _i.e._ not human-readable, which is done to save storage space. Indeed classical MD simulations can generate very large trajectories, and storage can become an issue. Moreover, the DCD file contains only the coordinates and no information about the atoms' type. Hence, we have to use an auxiliary file to retrieve that info, in this case the PDB files. This approach is also normal to many visualisation tools.

In [3]:
sys = md.Universe("mix.pdb", 'trajectory.1.dcd')
# md.topology.guessers.guess_atom_element("Ar")
nFrames = len(sys.trajectory)
print("Number of Frames : %4i" % nFrames)

FileNotFoundError: [Errno 2] No such file or directory: 'mix.pdb'

With MDAnalysis it is very easy to select a group of atoms, which can then be used to for the analysis.

In [ ]:
atoms = sys.select_atoms("all")
numberOfAtoms = len(atoms.positions)
print(" Number of Atoms : %4i" % numberOfAtoms)

Although we don't need the have the atomic charges for our analysis, we need to define them this to avoid errors later

In [ ]:
sys.add_TopologyAttr('charge', [0]*numberOfAtoms)

It is then easy to access information frame by frame by looping over the _system trajectory_ object. For example we can print the cell dimensions and the coordinates of the first atom.

In [ ]:
for iFrame in sys.trajectory:
    boxSize = np.array(sys.dimensions[0:3])
    print("Frame : %2i" % iFrame.frame)
    print("Cell dimensions [Angstroms]       : %8.3f %8.3f %8.3f" % 
        (boxSize[X],boxSize[Y],boxSize[Z]) )
    print("Coordinates of atom 1 [Angstroms] : %8.3f %8.3f %8.3f" % 
         (atoms.positions[0,X],atoms.positions[0,Y],atoms.positions[0,Z]) )
    print("---")
    if iFrame.frame == 2: break

We can then visualise the trajectory using nglview, which can directly read the MDAnalysis _system_.

In [ ]:
view = ng.show_mdanalysis(sys, gui=True)
view.center()
view.representations = [
    {"type": "spacefill", "params": {"sele": "all"}},
    {"type": "unitcell", "params": {"sele": "all"}}
]
view.camera = 'orthographic'
view

Another type of analysis that we ca do is to compute the density profile along the crystallographic directions. This can be readily done using the **lineardensity** function in the MDAnalysis package. 

In [ ]:
from MDAnalysis.analysis import lineardensity
ldens = lineardensity.LinearDensity(atoms,binsize=.2,verbose=True)
dmap = ldens.run()

x = list(np.arange(0,dmap.dimensions[X],dmap.binsize))

nx = len(x)-1

print(nx,ny,nz)
fig = plt.figure()
ax = fig.gca()

ax.plot(x[0:nx],dmap.results.x.pos[0:nx])

plt.show()

Alternatively, we could compute the atoms' density manually by looping over the frames and use the Numpy histogram function.

In [ ]:
histo = np.zeros(180, dtype=float)
for iFrame in sys.trajectory:
    y, x = np.histogram(atoms.positions[:,X], bins=180, range=(0.,180.))
    histo[:] = histo[:] + y
histo /= nFrames

fig = plt.figure()
ax = fig.gca()
ax.plot(x[0:-1],histo)
plt.show()